In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
9,application_1584073442397_0010,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-9>

In [7]:
sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version
-------------------------- -------
beautifulsoup4             4.8.1  
boto                       2.49.0 
jmespath                   0.9.4  
lxml                       4.4.2  
mysqlclient                1.4.6  
nltk                       3.4.5  
nose                       1.3.4  
numpy                      1.14.5 
pip                        20.0.2 
py-dateutil                2.2    
python36-sagemaker-pyspark 1.2.6  
pytz                       2019.3 
PyYAML                     3.11   
setuptools                 46.0.0 
six                        1.13.0 
soupsieve                  1.9.5  
wheel                      0.34.2 
windmill                   1.6

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession
import warnings; warnings.simplefilter('ignore')
import datetime

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# config spark to build context

spark=SparkSession.builder.appName("NYC.TLC-Green-Car-Data-ETL").getOrCreate()
spark.conf.set("spark.sql.session.timeZone", "America/New_York")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
#read in our raw dataset
df = spark.read.option('header', 'true') \
    .option('mode', 'FAILFAST') \
    .option('timeStampFormat', 'yyyy-MM-dd HH:mm:ss') \
    .option('columnNameOfCorruptRecord', 'error') \
    .csv('s3://data-etl-o-original-raw/green/*.csv') \
    .filter(year('lpep_pickup_datetime') == 2019) \
    .withColumn('duration', unix_timestamp('lpep_dropoff_datetime') - unix_timestamp('lpep_pickup_datetime')) \
    .withColumn('minute_rate',when(col('total_amount') <= 0.001,0).otherwise(col('total_amount')/col('duration') * 60)) \
    .withColumn('average_speed',when(col('trip_distance') < 0.001,0).otherwise(col('trip_distance')/col('duration') * 60 * 60)) \
    .drop('store_and_fwd_flag') \
    .drop('ehail_fee') \
    .drop('congestion_surcharge')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
## check schema structure
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- VendorID: string (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- trip_type: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- minute_rate: double (nullable = true)
 |-- average_speed: double (nullable = true)

In [67]:
# do some test
tmp = df.select(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'duration','total_amount','trip_distance','minute_rate','average_speed'])
tmp.show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+---------------------+--------+------------+-------------+--------------------+-------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|duration|total_amount|trip_distance|         minute_rate|      average_speed|
+--------+--------------------+---------------------+--------+------------+-------------+--------------------+-------------------+
|       2| 2019-01-01 00:10:16|  2019-01-01 00:16:32|     376|         7.3|          .86|  1.1648936170212767|   8.23404255319149|
|       2| 2019-01-01 00:27:11|  2019-01-01 00:31:38|     267|         5.8|          .66|   1.303370786516854|  8.898876404494382|
|       2| 2019-01-01 00:46:20|  2019-01-01 01:04:54|    1114|       19.71|         2.68|  1.0615798922800719|  8.660682226211849|
|       2| 2019-01-01 00:19:06|  2019-01-01 00:39:43|    1237|        19.3|         4.53|  0.9361358124494745| 13.183508488278092|
|       2| 2019-01-01 00:12:35|  2019-01-01 00:19:09|     394|         7.8|        

In [10]:
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'payment_type', 'trip_type', 'duration', 'minute_rate', 'average_speed']

In [69]:
## fix colums with regular names
for c in df.columns:
    df = df.withColumnRenamed(c, c.replace(' ', '_'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
## polish data

## remove data which is tooooo small and ilegal value
df = df.filter(df['trip_distance'] >= 0.001)

## round minute_rate and average_speed to scale 3 

df = df.withColumn('minute_rate', round(df['minute_rate'],4))

df = df.withColumn('average_speed', round(df['average_speed'],4))

# rename 'VendorID'
# df = df.withColumnRenamed('VendorID', 'vendor_id')

df.show(10)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+---------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+---------+--------+-----------+-------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|trip_type|duration|minute_rate|average_speed|
+--------+--------------------+---------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+---------+--------+-----------+-------------+
|       2| 2019-01-01 00:10:16|  2019-01-01 00:16:32|         1|          97|          49|              2|          .86|          6|  0.5|    0.5|         0|           0|                  0.3|         7.3|   

In [12]:
tmp = df.select(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'duration','total_amount','trip_distance','minute_rate','average_speed'])
tmp.show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+---------------------+--------+------------+-------------+-----------+-------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|duration|total_amount|trip_distance|minute_rate|average_speed|
+--------+--------------------+---------------------+--------+------------+-------------+-----------+-------------+
|       2| 2019-01-01 00:10:16|  2019-01-01 00:16:32|     376|         7.3|          .86|     1.1649|        8.234|
|       2| 2019-01-01 00:27:11|  2019-01-01 00:31:38|     267|         5.8|          .66|     1.3034|       8.8989|
|       2| 2019-01-01 00:46:20|  2019-01-01 01:04:54|    1114|       19.71|         2.68|     1.0616|       8.6607|
|       2| 2019-01-01 00:19:06|  2019-01-01 00:39:43|    1237|        19.3|         4.53|     0.9361|      13.1835|
|       2| 2019-01-01 00:12:35|  2019-01-01 00:19:09|     394|         7.8|         1.05|     1.1878|       9.5939|
|       2| 2019-01-01 00:47:55|  2019-01-01 01:00:01|     726|        14

In [13]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5779303

In [18]:
## Write Data Back to S3 using AWS API
currenttime = datetime.datetime.now()

df.write.option('compression', 'snappy') \
    .save( "s3://data-etl-o-target-0/result/green/green_car_data_processed_{date}/".format(date=currenttime.strftime('%Y%m%d%H%M%s')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:

#read in our raw dataset
dfz = spark.read.option('header', 'true') \
    .option('mode', 'FAILFAST') \
    .option('columnNameOfCorruptRecord', 'error') \
    .csv('s3://data-etl-o-original-raw/zone/nyc.tlc.taxi_zone_lookup.csv') \

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
dfz.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

265

In [84]:
dfz.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['LocationID', 'Borough', 'Zone', 'service_zone']

In [15]:
dfz.show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [88]:
dfz.select('Borough').distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+
|      Borough|
+-------------+
|       Queens|
|          EWR|
|      Unknown|
|     Brooklyn|
|Staten Island|
|    Manhattan|
|        Bronx|
+-------------+

In [89]:
dfz.show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [17]:
dfz.select('Borough').groupBy('Borough').agg({'Borough': 'count'}).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+--------------+
|      Borough|count(Borough)|
+-------------+--------------+
|       Queens|            69|
|          EWR|             1|
|      Unknown|             2|
|     Brooklyn|            61|
|Staten Island|            20|
|    Manhattan|            69|
|        Bronx|            43|
+-------------+--------------+

In [99]:
%%local
conda list

# packages in environment at /opt/conda:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                 conda_forge    conda-forge
_openmp_mutex             4.5                      1_llvm    conda-forge
alabaster                 0.7.12                     py_0    defaults
anaconda-client           1.7.2                    py37_0    defaults
anaconda-navigator        1.9.7                    py37_0    defaults
anaconda-project          0.8.4                      py_0    defaults
asn1crypto                1.3.0                    py37_0    defaults
astroid                   2.3.3                    py37_0    defaults
astropy                   4.0              py37h7b6447c_0    defaults
attrs                     19.3.0                     py_0    defaults
autovizwidget             0.13.1                     py_0    defaults
awscli                    1.18.3                   py37_0    conda-forge
awseditorssparkmonitoringwidget 1.0      

In [100]:
%%help

Magic,Example,Explanation
info,%%info,Outputs session information for the current Livy endpoint.
cleanup,%%cleanup -f,"Deletes all sessions for the current Livy endpoint, including this notebook's session. The force flag is mandatory."
delete,%%delete -f -s 0,Deletes a session by number for the current Livy endpoint. Cannot delete this kernel's session.
logs,%%logs,Outputs the current session's Livy logs.
configure,"%%configure -f{""executorMemory"": ""1000M"", ""executorCores"": 4}",Configure the session creation parameters. The force flag is mandatory if a session has already been created and the session will be dropped and recreated.Look at Livy's POST /sessions Request Body for a list of valid parameters. Parameters must be passed in as a JSON string.
spark,%%spark -o dfdf = spark.read.parquet('...,"Executes spark commands. Parameters: -o VAR_NAME: The Spark dataframe of name VAR_NAME will be available in the %%local Python context as a Pandas dataframe with the same name. -m METHOD: Sample method, either take or sample. -n MAXROWS: The maximum number of rows of a dataframe that will be pulled from Livy to Jupyter. If this number is negative, then the number of rows will be unlimited. -r FRACTION: Fraction used for sampling."
display,%%display -n 100my_spark_df,"Displays Spark dataframe as a beautiful HTML table with horizontal and vertical scroll bars(if necessary). Parameters: -m METHOD: Sample method, either take or sample. -n MAXROWS: The maximum number of rows of a dataframe to display. If this number is negative, then the number of rows will be unlimited. -r FRACTION: Fraction used for sampling."
sql,%%sql -o tables -qSHOW TABLES,"Executes a SQL query against the variable sqlContext (Spark v1.x) or spark (Spark v2.x). Parameters: -o VAR_NAME: The result of the SQL query will be available in the %%local Python context as a Pandas dataframe. -q: The magic will return None instead of the dataframe (no visualization). -m, -n, -r are the same as the %%spark parameters above."
local,%%locala = 1,All the code in subsequent lines will be executed locally. Code must be valid Python code.


In [101]:
tmp = df.select(['VendorID','PULocationID', 'DOLocationID', 'passenger_count', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'duration','total_amount','trip_distance','minute_rate','average_speed'])
tmp.show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------------+------------+---------------+--------------------+---------------------+--------+------------+-------------+-----------+-------------+
|VendorID|PULocationID|DOLocationID|passenger_count|lpep_pickup_datetime|lpep_dropoff_datetime|duration|total_amount|trip_distance|minute_rate|average_speed|
+--------+------------+------------+---------------+--------------------+---------------------+--------+------------+-------------+-----------+-------------+
|       2|          97|          49|              2| 2019-01-01 00:10:16|  2019-01-01 00:16:32|     376|         7.3|          .86|      1.165|        8.234|
|       2|          49|         189|              2| 2019-01-01 00:27:11|  2019-01-01 00:31:38|     267|         5.8|          .66|      1.303|        8.899|
|       2|         189|          17|              2| 2019-01-01 00:46:20|  2019-01-01 01:04:54|    1114|       19.71|         2.68|      1.062|        8.661|
|       2|          82|         258|              1|

In [104]:
df.groupBy('passenger_count').count().orderBy('count', ascending=False).limit(10).show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------------+------------+---------------+--------------------+---------------------+--------+------------+-------------+-----------+-------------+
|VendorID|PULocationID|DOLocationID|passenger_count|lpep_pickup_datetime|lpep_dropoff_datetime|duration|total_amount|trip_distance|minute_rate|average_speed|
+--------+------------+------------+---------------+--------------------+---------------------+--------+------------+-------------+-----------+-------------+
|       2|          97|          49|              2| 2019-01-01 00:10:16|  2019-01-01 00:16:32|     376|         7.3|          .86|      1.165|        8.234|
|       2|          49|         189|              2| 2019-01-01 00:27:11|  2019-01-01 00:31:38|     267|         5.8|          .66|      1.303|        8.899|
|       2|         189|          17|              2| 2019-01-01 00:46:20|  2019-01-01 01:04:54|    1114|       19.71|         2.68|      1.062|        8.661|
|       2|          82|         258|              1|

In [19]:
toplo = df.groupBy('PULocationID').count().orderBy('count', ascending=False).limit(10)
toplo.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------+
|PULocationID| count|
+------------+------+
|          74|457848|
|          75|368752|
|          41|333623|
|          82|259303|
|           7|244131|
+------------+------+
only showing top 5 rows

In [20]:
toplo2 = df.groupBy('PULocationID','DOLocationID').count().orderBy('count', ascending=False).limit(10)
toplo2.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+-----+
|PULocationID|DOLocationID|count|
+------------+------------+-----+
|          75|          74|72275|
|           7|           7|67275|
|          74|          75|61992|
|          41|          42|56941|
|          95|          95|52950|
+------------+------------+-----+
only showing top 5 rows

In [21]:
dfz.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows

In [22]:
top_5_loc_names = toplo2.join(dfz, toplo2['PULocationID'].eqNullSafe(dfz['LocationID']),'left') \
            .withColumnRenamed('Borough','FromLocation') \
            .drop('LocationID') \
            .orderBy('count', ascending=False)
top_5_loc_names.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+-----+------------+-----------------+------------+
|PULocationID|DOLocationID|count|FromLocation|             Zone|service_zone|
+------------+------------+-----+------------+-----------------+------------+
|          75|          74|72275|   Manhattan|East Harlem South|   Boro Zone|
|           7|           7|67275|      Queens|          Astoria|   Boro Zone|
|          74|          75|61992|   Manhattan|East Harlem North|   Boro Zone|
|          41|          42|56941|   Manhattan|   Central Harlem|   Boro Zone|
|          95|          95|52950|      Queens|     Forest Hills|   Boro Zone|
|          82|         129|49679|      Queens|         Elmhurst|   Boro Zone|
|          41|          41|43924|   Manhattan|   Central Harlem|   Boro Zone|
|          74|          41|43754|   Manhattan|East Harlem North|   Boro Zone|
|          74|          74|43477|   Manhattan|East Harlem North|   Boro Zone|
|         129|         129|43268|      Queens|  Jackson Heights|

In [114]:
%%help

Magic,Example,Explanation
info,%%info,Outputs session information for the current Livy endpoint.
cleanup,%%cleanup -f,"Deletes all sessions for the current Livy endpoint, including this notebook's session. The force flag is mandatory."
delete,%%delete -f -s 0,Deletes a session by number for the current Livy endpoint. Cannot delete this kernel's session.
logs,%%logs,Outputs the current session's Livy logs.
configure,"%%configure -f{""executorMemory"": ""1000M"", ""executorCores"": 4}",Configure the session creation parameters. The force flag is mandatory if a session has already been created and the session will be dropped and recreated.Look at Livy's POST /sessions Request Body for a list of valid parameters. Parameters must be passed in as a JSON string.
spark,%%spark -o dfdf = spark.read.parquet('...,"Executes spark commands. Parameters: -o VAR_NAME: The Spark dataframe of name VAR_NAME will be available in the %%local Python context as a Pandas dataframe with the same name. -m METHOD: Sample method, either take or sample. -n MAXROWS: The maximum number of rows of a dataframe that will be pulled from Livy to Jupyter. If this number is negative, then the number of rows will be unlimited. -r FRACTION: Fraction used for sampling."
display,%%display -n 100my_spark_df,"Displays Spark dataframe as a beautiful HTML table with horizontal and vertical scroll bars(if necessary). Parameters: -m METHOD: Sample method, either take or sample. -n MAXROWS: The maximum number of rows of a dataframe to display. If this number is negative, then the number of rows will be unlimited. -r FRACTION: Fraction used for sampling."
sql,%%sql -o tables -qSHOW TABLES,"Executes a SQL query against the variable sqlContext (Spark v1.x) or spark (Spark v2.x). Parameters: -o VAR_NAME: The result of the SQL query will be available in the %%local Python context as a Pandas dataframe. -q: The magic will return None instead of the dataframe (no visualization). -m, -n, -r are the same as the %%spark parameters above."
local,%%locala = 1,All the code in subsequent lines will be executed locally. Code must be valid Python code.


In [23]:
top_5_loc_names.createOrReplaceTempView("top_5_loc_names_views")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
%%sql -o top_5_loc_names -n -1
select 'PULocationID','DOLocationID','count' from top_5_loc_names_views
group by 'PULocationID','DOLocationID'
ORDER BY count Desc

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [31]:
import os
all_files_in_cur_dir = os.listdir('.') 
csvfileslist = list(filter(lambda x:(x[-6:] == '.ipynb'),all_files_in_cur_dir))
csvfileslist

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[]

In [34]:
import os
# csvfileslist = list(os.listdir('.'))
csvfileslist
os.listdir('.')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['var', 'lib64', 'emr', 'home', 'lost+found', 'sbin', 'selinux', 'lib', 'srv', 'device-names.json', 'media', 'tmp', 'mnt', 'run', 'cgroup', 'bin', 'etc', 'dev', 'mnt1', '.autofsck', 'boot', 'opt', 'usr', 'proc', 'sys', 'root', 'local', '.autorelabel']

In [44]:
import urllib 
urllib.request.urlretrieve("https://resources.lendingclub.com/LoanStats3a.csv.zip", "/tmp/LoanStats3a.csv.zip")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('/tmp/LoanStats3a.csv.zip', <http.client.HTTPMessage object at 0x7f604f355400>)

In [46]:
import zipfile
urllib.request.urlretrieve("https://s3.amazonaws.com/nyc-tlc/misc/taxi_zones.zip", "/tmp/taxi_zones.zip")
with zipfile.ZipFile("/tmp/taxi_zones.zip","r") as zip_ref:
    zip_ref.extractall("/tmp/shape")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
os.listdir('/tmp/shape')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['taxi_zones.prj', 'taxi_zones.dbf', 'taxi_zones.shp', 'taxi_zones.shp.xml', 'taxi_zones.shx', 'taxi_zones.sbn', 'taxi_zones.sbx']

In [54]:
#read in our raw dataset
shpt1 = sc.binaryFiles('s3://data-etl-o-original-raw/zone/taxi_zones_shape.zip')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
shpt1.saveAsTextFile('/tmp/taxi_zones_shape_1.zip')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
tmpfile = list(filter(lambda x:(x[-4:] == '.zip'),os.listdir('/tmp')))
tmpfile

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['LoanStats3a.csv.zip', 'taxi_zones.zip']

In [57]:
shpt2 = sc.textFile('s3://data-etl-o-original-raw/zone/taxi_zones_shape.zip')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
shpt2.saveAsTextFile('/tmp/taxi_zones_shape_2.zip')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
tmpfile = list(filter(lambda x:(x[-4:] == '.zip'),os.listdir('/tmp')))
tmpfile

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['LoanStats3a.csv.zip', 'taxi_zones.zip']